In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# Example DataFrame
df = pd.DataFrame({
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'target': np.random.rand(100)  # Replace with your target variable
})

# Define features and target
X = df[['feature1', 'feature2']]  # Replace with your features
y = df['target']  # Replace with your target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
from xgboost import XGBRegressor

# Initialize the model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [3]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions
predictions = model.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")


Mean Squared Error: 0.13
R-squared: -0.55


In [4]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, cv=3)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)


Best parameters: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100}


In [5]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(XGBRegressor(), X, y, cv=5)

print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV score: {cv_scores.mean()}")


Cross-validation scores: [-0.00489781 -0.87013006 -0.9652284  -0.5760747  -0.4447457 ]
Mean CV score: -0.572215334198934


In [6]:
import joblib

# Save the model
joblib.dump(model, 'xgboost_model.pkl')


['xgboost_model.pkl']

In [7]:
print(X_train.dtypes)


feature1    float64
feature2    float64
dtype: object


In [8]:
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        print(f"Column '{col}' has non-numeric values")


In [9]:
if 'numeric_column' in X_train.columns:
    X_train['numeric_column'] = X_train['numeric_column'].apply(clean_numeric)
if 'numeric_column' in X_test.columns:
    X_test['numeric_column'] = X_test['numeric_column'].apply(clean_numeric)
    

In [10]:
def clean_numeric(value):
    try:
        return float(value.split()[0])
    except ValueError:
        return np.nan  # Handle missing or non-numeric values

# Check if the column exists in X_train and X_test
if 'numeric_column' in X_train.columns:
    X_train['numeric_column'] = X_train['numeric_column'].apply(clean_numeric)

if 'numeric_column' in X_test.columns:
    X_test['numeric_column'] = X_test['numeric_column'].apply(clean_numeric)


In [11]:
X_train.fillna(0, inplace=True)  # Example: fill missing values with 0
X_test.fillna(0, inplace=True)


In [12]:
# import pandas as pd
# import numpy as np
# from xgboost import XGBRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# # Step 1: Load the datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Display the first few rows of the datasets for reference
# print(train_df.head())
# print(test_df.head())

# # Identify non-numeric columns
# non_numeric_cols = ['image_link', 'entity_name']  # Example of non-numeric columns

# # Convert non-numeric columns to numeric using Label Encoding
# label_encoders = {}
# for col in non_numeric_cols:
#     le = LabelEncoder()
#     # Fit the encoder only on the training data
#     le.fit(train_df[col].astype(str))
#     # Transform both training and test data
#     train_df[col] = le.transform(train_df[col].astype(str))
#     # Ensure all possible labels are known by the encoder
#     test_df[col] = test_df[col].astype(str)
#     # Use a mapping to handle unseen labels in test data
#     test_df[col] = test_df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
#     label_encoders[col] = le

# # Handle numeric columns with non-numeric values
# def clean_numeric(value):
#     try:
#         return float(value.split()[0])
#     except ValueError:
#         return np.nan

# numeric_columns = ['some_numeric_column']  # Replace with actual numeric columns
# for col in numeric_columns:
#     train_df[col] = train_df[col].apply(clean_numeric)
#     test_df[col] = test_df[col].apply(clean_numeric)

# # Handle missing values if necessary
# train_df.fillna(0, inplace=True)
# test_df.fillna(0, inplace=True)

# # Prepare the data for training
# X = train_df.drop(columns=['entity_value'])
# y = train_df['entity_value']

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=44)

# # Initialize and train the model
# model = XGBRegressor()
# model.fit(X_train, y_train)

# # Step 3: Generate predictions
# X_test = test_df.drop(columns=['id'])  # Dropping 'id' if it's not a feature
# predictions = model.predict(X_test)

# # Convert predictions to a DataFrame
# predictions_df = pd.DataFrame(predictions, columns=['entity_value'])

# # Ensure predictions are in the same format as sample_test_out.csv
# # For this example, we'll assume predictions need to be formatted as strings
# predictions_df['entity_value'] = predictions_df['entity_value'].apply(lambda x: f"{x:.2f}")

# # Step 4: Prepare the submission file
# submission_df = test_df[['id']].copy()
# submission_df['entity_value'] = predictions_df['entity_value']
# submission_df.to_csv('test_out.csv', index=False)


In [13]:
# import pandas as pd

# def evaluate_f1_score(ground_truth_file, prediction_file):
#     # Load the ground truth and prediction files
#     gt_df = pd.read_csv(ground_truth_file)
#     pred_df = pd.read_csv(prediction_file)

#     # Ensure both files have the same key to merge on, e.g., 'index' or 'image_link'
#     common_key = 'index'  # Replace with the actual common key
    
#     # Merge ground truth and predictions based on the common key
#     merged_df = pd.merge(gt_df[[common_key, 'entity_value']], pred_df[[common_key, 'prediction']], on=common_key, how='inner')

#     # Initialize counts for TP, FP, FN, and TN
#     TP = FP = FN = TN = 0

#     # Iterate through each row to classify predictions
#     for _, row in merged_df.iterrows():
#         GT = str(row['entity_value']).strip()  # Ground Truth
#         OUT = str(row['prediction']).strip()   # Model Prediction

#         if OUT != "" and GT != "" and OUT == GT:
#             TP += 1  # True Positive
#         elif OUT != "" and GT != "" and OUT != GT:
#             FP += 1  # False Positive
#         elif OUT != "" and GT == "":
#             FP += 1  # False Positive (prediction when none expected)
#         elif OUT == "" and GT != "":
#             FN += 1  # False Negative
#         elif OUT == "" and GT == "":
#             TN += 1  # True Negative

#     # Calculate Precision, Recall, and F1 Score
#     precision = TP / (TP + FP) if (TP + FP) > 0 else 0
#     recall = TP / (TP + FN) if (TP + FN) > 0 else 0
#     f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#     # Print the results
#     print(f"True Positives (TP): {TP}")
#     print(f"False Positives (FP): {FP}")
#     print(f"False Negatives (FN): {FN}")
#     print(f"True Negatives (TN): {TN}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1 Score: {f1_score:.4f}")

#     return f1_score

# if __name__ == "__main__":
#     ground_truth_file = 'test_with_entity_value.csv'  # Replace with your ground truth CSV file path
#     prediction_file = 'test_predictions.csv'  # Replace with your prediction CSV file path
    
#     # Evaluate the F1 score
#     evaluate_f1_score(ground_truth_file, prediction_file)


In [14]:
import pandas as pd

# Example: Convert a continuous target variable into binary categories
threshold = y.median()  # or any other threshold you consider
y_discrete = (y > threshold).astype(int)


In [15]:
from sklearn.preprocessing import LabelEncoder

# Example: Convert categorical labels to numeric labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [16]:
import pandas as pd

# Check the unique values of y
print(pd.Series(y).unique())


[0.12343733 0.37654417 0.94764998 0.34559432 0.98943367 0.84236599
 0.86462493 0.38299823 0.90179911 0.5023176  0.79024573 0.07215671
 0.36792952 0.13000841 0.6726842  0.00202025 0.44812766 0.70876703
 0.63258448 0.2244238  0.53919506 0.72225243 0.9753242  0.84248923
 0.2989749  0.75571058 0.25125338 0.00194727 0.6785728  0.20666602
 0.56033686 0.03538249 0.60666591 0.18857153 0.28769726 0.49168268
 0.79719661 0.69250547 0.442872   0.2916748  0.83205352 0.62708059
 0.22043033 0.77165697 0.42091428 0.83143822 0.46641491 0.88793969
 0.90021545 0.55660535 0.36518162 0.63269906 0.1134044  0.33401311
 0.69991535 0.24494195 0.32925849 0.16348379 0.60354538 0.90353079
 0.37911655 0.3725102  0.79871092 0.98838718 0.38618883 0.32569196
 0.20483361 0.52652274 0.19805605 0.97737599 0.30488316 0.69960073
 0.3168252  0.27391481 0.89054993 0.35668795 0.17954323 0.51518028
 0.81601894 0.98704569 0.04290321 0.50107395 0.68161787 0.18603143
 0.12731583 0.22327616 0.11088555 0.96991919 0.71474854 0.8350

In [17]:
# Example: Convert continuous target to binary classes
threshold = y.median()  # You can use a different threshold if needed
y_discrete = (y > threshold).astype(int)


In [18]:
import numpy as np
import pandas as pd

# Assuming y_train is your target variable
unique_classes = np.unique(y_train)
print(f"Unique classes in y_train: {unique_classes}")
print(f"Number of unique classes: {len(unique_classes)}")


Unique classes in y_train: [0.00194727 0.00202025 0.07215671 0.11088555 0.1134044  0.12731583
 0.13000841 0.16348379 0.19805605 0.20483361 0.20666602 0.22043033
 0.22327616 0.2244238  0.24494195 0.2488842  0.25125338 0.28605586
 0.28769726 0.2989749  0.3168252  0.32569196 0.32925849 0.34559432
 0.35668795 0.36518162 0.3725102  0.37654417 0.37911655 0.38299823
 0.38618883 0.442872   0.44812766 0.46641491 0.48447214 0.49168268
 0.50107395 0.5023176  0.52652274 0.53919506 0.55660535 0.60354538
 0.60666591 0.62708059 0.63269906 0.65948032 0.6726842  0.67654317
 0.6785728  0.68161787 0.69250547 0.69960073 0.69991535 0.70876703
 0.71474854 0.72225243 0.74899334 0.75571058 0.77165697 0.79719661
 0.79871092 0.81601894 0.83205352 0.83503001 0.84236599 0.84248923
 0.86462493 0.8842671  0.88793969 0.89054993 0.90021545 0.90179911
 0.90353079 0.94720301 0.94764998 0.96094561 0.96991919 0.97737599
 0.98704569 0.98838718]
Number of unique classes: 80


In [19]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check class distribution
print("Training set class distribution:", pd.Series(y_train).value_counts())
print("Test set class distribution:", pd.Series(y_test).value_counts())


Training set class distribution: target
0.072157    1
0.676543    1
0.988387    1
0.903531    1
0.484472    1
           ..
0.835030    1
0.329258    1
0.960946    1
0.699915    1
0.632699    1
Name: count, Length: 70, dtype: int64
Test set class distribution: target
0.186031    1
0.334013    1
0.502318    1
0.659480    1
0.832054    1
0.002020    1
0.977376    1
0.220430    1
0.251253    1
0.714749    1
0.244942    1
0.035382    1
0.367930    1
0.515180    1
0.179543    1
0.989434    1
0.201194    1
0.188572    1
0.273915    1
0.560337    1
0.632584    1
0.123437    1
0.790246    1
0.042903    1
0.975324    1
0.291675    1
0.420914    1
0.831438    1
0.304883    1
0.316825    1
Name: count, dtype: int64


In [20]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Assuming X and y are your features and target
X_train, X_test, y_train, y_test = train_test_split(X, y_discrete, test_size=0.3, random_state=42)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Initialize and train the model
model = LogisticRegression(random_state=42)
model.fit(X_train_balanced, y_train_balanced)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[10  8]
 [ 6  6]]
              precision    recall  f1-score   support

           0       0.62      0.56      0.59        18
           1       0.43      0.50      0.46        12

    accuracy                           0.53        30
   macro avg       0.53      0.53      0.52        30
weighted avg       0.55      0.53      0.54        30



In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Define your custom evaluation metrics
def compute_custom_metrics(y_true, y_pred):
    tp = np.sum((y_pred != "") & (y_true != "") & (y_pred == y_true))
    fp = np.sum((y_pred != "") & (y_true != "") & (y_pred != y_true))
    fn = np.sum((y_pred == "") & (y_true != ""))
    tn = np.sum((y_pred == "") & (y_true == ""))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Example usage with your predictions
precision, recall, f1 = compute_custom_metrics(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Prepare and save the submission file
submission_df = pd.DataFrame({
    'Index': X_test.index,  # Make sure to use the actual index column from your dataset
    'Prediction': y_pred
})

submission_df.to_csv('test_out.csv', index=False)


Precision: 0.5333
Recall: 1.0000
F1 Score: 0.6957
